In [1]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
TASK = 'hyperkalemia'

In [2]:
TASK = 'hypoglycemia'

In [3]:
demo_df = pd.read_parquet(f'data/lab_{TASK}/{TASK}_demo.parquet')
label_df = pd.read_parquet(f'data/lab_{TASK}/{TASK}_label2.parquet')

In [4]:
AGES = [0, 20, 30, 40, 50, 60, 70, 80, 90]
AGE_LABELS = [f'Age/{AGES[i]}-{AGES[i+1]}' for i in range(0, len(AGES)-1)]
def age_str(x):
    ind = np.searchsorted(AGES, x, 'right')
    return AGE_LABELS[ind-1]

In [5]:
demo_df['race_str'] = 'Race/' + demo_df['race']
demo_df['gender_str'] = 'Gender/' + demo_df['gender']
demo_df['ethnicity_str'] = 'Ethnicity/' + demo_df['ethnicity']
demo_df['age_str'] = demo_df['age'].apply(age_str)

In [6]:
demo_df.to_parquet(f'data/lab_{TASK}/{TASK}_demo.parquet')

In [7]:
label_df['Time'] = label_df.apply(lambda x: x['prediction_time'] if x['value'] else x['Sample_time'], axis=1)

In [8]:
label_df.to_parquet(f'data/lab_{TASK}/{TASK}_label2.parquet')

EVENTVAL

In [9]:
IGNORE_VALS = ['NONE SEEN', 'See Note', 'See Below', 'See note', 'See scanned results', 'Not done', 'Comment:', 'Comment',
                'SEE COMMENT', '(NOTE)', 'Detailed information on file in HIS.', 'SEE BELOW','SEE TEXT', 'SEE NOTE', 'SEE COMMENTS',
                'Unable to interpret',  'See Comment', 'Note', 'note', 'tnp', 'not given', 'scanned:  shc/epic media manager', 
                'not applicable', 'indeterminate', 'random', 'equivocal', '-', 'final report']
IGNORE_VALS = [v.lower() for v in IGNORE_VALS]

In [10]:
def convert_value(x):
    x = x.lower()
    if x == 'non reactive' or x == 'non-reactive' or x == 'nonreactive':
        return 'non-reactive'
    elif x == 'not detected' or x == 'none detected':
        return 'not detected'
    elif x == 'occ.':
        return 'occasional'
    elif x == 'dnr':
        return 'normal'
    elif x == 'intermed.':
        return 'intermediate'
    else:
        return x

In [11]:
def try_parse_float(x):
    if isinstance(x, str):
        try:
            return float(x)
        except ValueError:
            if 'pos' in x.lower() or '+' in x.lower(): return 1.0
            elif 'neg' in x.lower(): return 0.0
            else:
                return x
    return x

In [12]:
def str_bucket_ind(event, val, d):
    bucket = d[event]
    if type(bucket) == str: 
        return 0
    ind = np.where(bucket == val)[0][0]
    return ind

In [13]:
def bucket_eventval(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return buckets
    ind = np.searchsorted(buckets, val, side='right')
    if ind == len(buckets):
        eventval = f"{event}|{buckets[ind-1]}-"
    else:
        eventval = f"{event}|{buckets[ind-1]}-{buckets[ind]}"
    return eventval

In [14]:
def bucket_ind(event, val, d):
    buckets = d[event]
    if type(buckets) == str: 
        return 0
    ind = np.searchsorted(buckets, val, side='right')
    return ind

In [15]:
with open('next_token/buckets.pkl', 'rb') as file:
    buckets = pickle.load(file)

with open('next_token/str_buckets.pkl', 'rb') as file:
    str_buckets = pickle.load(file)

In [16]:
df = pd.read_parquet(f'data/lab_{TASK}/{TASK}2.parquet', columns=['patient_id', 'start', 'code', 'value'])
df['value'] = df['value'].apply(try_parse_float)

In [17]:
float_df = df[df['value'].apply(type) == float]
str_df = df[(df['value'].apply(type) == str)]
none_df = df[df['value'].isna()]

In [18]:
float_df['eventval'] = float_df.apply(lambda x: bucket_eventval(x['code'], x['value'], buckets), axis=1)
float_df['bucket'] = float_df.apply(lambda x: bucket_ind(x['code'], x['value'], buckets), axis=1)

/tmp/ipykernel_72487/3018597392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  float_df['eventval'] = float_df.apply(lambda x: bucket_eventval(x['code'], x['value'], buckets), axis=1)
/tmp/ipykernel_72487/3018597392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  float_df['bucket'] = float_df.apply(lambda x: bucket_ind(x['code'], x['value'], buckets), axis=1)


In [19]:
none_code = none_df[none_df['code'].apply(lambda x: type(buckets[x]) == str)]
none_df = none_df[~(none_df['code'].isin(none_code['code']))]
none_df = none_df[none_df['code'].isin(str_buckets.keys())]
none_str = none_df[none_df['code'].apply(lambda x: type(str_buckets[x]) == str)]

none_df = pd.concat([none_code, none_str], ignore_index=True)
none_df['eventval'] = none_df['code']
none_df['bucket'] = 0

In [20]:
str_df = str_df[~(str_df['value'].str.lower().isin(IGNORE_VALS))]
str_df['value'] = str_df['value'].apply(convert_value)
str_df['eventval'] = (str_df['code'] + '|' + str_df['value'])
str_df['bucket'] = str_df.apply(lambda x: str_bucket_ind(x['code'], x['value'], str_buckets), axis=1)

In [21]:
df = pd.concat([float_df, str_df, none_df], ignore_index=True)

In [22]:
df['value'] = df['value'].astype(str)

In [23]:
df.to_parquet(f'data/lab_{TASK}/{TASK}2.parquet')